*Ce notebook est distribué par Devlog sous licence Creative Commons - Attribution - Pas d’Utilisation Commerciale - Partage dans les Mêmes Conditions. La description complète de la license est disponible à l'adresse web http://creativecommons.org/licenses/by-nc-sa/4.0/.*

# Initiation Python - Objets 4/6 : L'héritage entre classes

## Généralités 

Python permet d'exprimer une relation "est une sorte de" entre deux classes. Si `B` est une sorte de `A`, on s'attend à ce que les instances de `B` possèdent les attributs (données et méthodes) de cette classe, mais aussi tous ceux de `A`, puisque ces objets sont aussi une sorte de `A`. On dit que `B` hérite de `A`, ou dérive de `A`. On peut aussi qualifier `A` de "classe mère", de "classe de base", de "super-classe", et `B` de "classe dérivée", de "classe fille", de "sous-classe" de `A`.

Pour hériter d'une classe mère, il suffit d’indiquer son nom entre parenthèses après le nom de la classe fille. La classe fille a accès à tous les attributs de la classe mère. En d'autres termes, quand on écrit `obj.x`, l'interpréteur cherche `x` dans l'instance, puis dans sa classe, puis dans la classe mère de cette classe, etc.

In [ ]:
class A:
    def x(self): return "A"
    def y(self): return "A"
    
class B(A):
    def z(self): return "B"
    
b = B()

print(b.x(), b.y(), b.z())

<img src="img-objets/heritage/heritage-simple.png" width=167px>

## Surcharge

Une classe dérivée peut ajouter de nouveaux attributs, qu'il s'agisse de données ou de méthodes. Elle peut aussi redéfinir un des attributs de la classe de base, auquel cas, pour les instances de la classe dérivée, c'est la nouvelle définition qui sera trouvée la première, et utilisée. On parle de "surcharge".

In [ ]:
class A:
    def x(self): return "A"
    def y(self): return "A"
    def z(self): return "A"
    
class B(A):
    def x(self): return "B"
    def t(self): return "B"
    
b = B()

print(b.x(), b.y(), b.z(), b.t())

<img src="img-objets/heritage/heritage-surcharge.png" width=192px>

## Héritage multiple

Une classe peut hériter de plusieurs autres en même temps (on donne alors une liste de noms séparés par des virgules). L'ordre de gauche à droite des classes parentes est respecté lorsque l'interpréteur Python recherche un nom (donnée ou méthode).

In [ ]:
class A1:
    def x(self): return "A1"
    
class A2:
    def x(self): return "A2"
    
class B(A1, A2):
    pass

b = B()
print(b.x())

<img src="img-objets/heritage/heritage-multiple.png" width=201px>

## Arborescence d'héritage

En plus de l'héritage multiple, chaque classe de base peut à son tour hériter d'une autre classe, etc. L'ensemble des classes dont hérite une instance forme ainsi une arborescence parfois complexe. Dans cette arborescence, la recherche d'un attribut se fait de bas en haut et de gauche à droite, en profondeur d'abord, mais en faisant en sorte qu'une classe de base ne soit jamais explorée avant que toutes ses dérivées le soient. Ainsi, dans l'exemple ci-dessous, `A` est prioritaire sur `C`, mais pas sur `B2` :

In [ ]:
class A:
    def x(self): return "A"
    def y(self): return "A"
    def z(self): return "A"
    
class B1(A):
    pass
    
class B2(A):
    def x(self): return "B2"
    
class C:
    def y(self): return "C"
    def z(self): return "C"
    
class D(B1, B2, C):
    z = C.z

d = D()
print(d.x(), d.y(), d.z())

<img src="img-objets/heritage/heritage-arborescence.png" width=305px>

L'ordre de parcours des classes est appelé le **MRO** (Method Resolution Order).

On voit aussi, dans l'exemple ci-dessus, que l'on peut explicitement copier la méthode `z` de `C` dans `D`. Ainsi, tout appel à la méthode `z` pour une instance de `D` se trouve redirigé vers la méthode de `C` (au lieu de `A`).

## Chaque recherche d'attribut est indépendante et repart de `self`

Lorsque j'appelle une méthode m1 de l'objet obj, cette méthode est recherchée dans l'arborescence de classes de obj. Si, à son tour, m1 appelle une autre méthode m2, la recherche de m2 repart de obj (et pas de la classe de m1). En C++, on dirait que toutes les méthodes sont virtuelles et toutes les variables polymorphiques.

In [ ]:
class A:
    def x(self): return "A"
    def y(self): return "A"
    def affichex(self): print( self.x() )
    
class B1(A):
    pass
    
class B2(A):
    def x(self): return "B2"
    
class C:
    def x(self): return "C"
    def affichey(self): print(self.y())
    
class D(B1, B2, C):
    pass

d = D()
d.affichex()
d.affichey()

<img src="img-objets/heritage/heritage-self.png" width=305px>

Si on transforme les méthodes `x()` et `y()` en attributs pseudo-privés, ces attributs ne sont visibles qu'aux méthodes de leurs classes respectives, et ces méthodes utiliseront exclusivement ces attributs.

In [ ]:
class A:
    def __x(self): return "A"
    def __y(self): return "A"
    def affichex(self): print(self.__x())
    
class B1(A):
    pass
    
class B2(A):
    def __x(self): return "B2"
    
class C:
    def __y(self): return "C"
    def affichey(self): print(self.__y())
    
class D(B1, B2, C):
    pass

d = D()
d.affichex()
d.affichey()

<img src="img-objets/heritage/heritage-pseudo-prive.png" width=316px>

## Réutiliser explicitement une méthode de base

Lorsqu'une méthode est redéfinie dans une classe dérivée, on dit qu'elle est surchargée. Plutôt que de réécrire toutes les instructions, il peut être utile de commencer par exécuter le code de la méthode de la classe de base. C'est particulièrement vrai pour les méthodes spéciales, telles que les constructeurs. Il est assez facile d'appeler explicitement une méthode de base en passant par le nom de la classe :

In [ ]:
class Fruit:
    def __init__(self, couleur, variete=''):
        self.__couleur = couleur
        self.__variete = variete
        
    def __str__(self):
        if (self.__variete!=''):
            return "{} {}".format(self.__variete, self.__couleur)
        else:
            return "fruit {}".format(self.__couleur)

class Pomme(Fruit):
    def __init__(self, couleur, variete='golden'):
        Fruit.__init__(self, couleur, variete)
        
    def __str__(self):
        return ("pomme " + Fruit.__str__(self))

    
p = Pomme('rouge')
p2 = Pomme('verte', 'grany')
print(p)
print(p2)

<img src="img-objets/heritage/heritage-pomme.png" width=183px>

On peut remplacer l'appel explicite à la classe de base par un appel à la fonction prédéfinie `super()`. Dans le cas d'un héritage simple, cela permet ensuite de revoir les classe de base sans avoir à corriger tous les appels directs aux méthodes de la classe de base.

In [ ]:
class Pomme(Fruit):
    def __init__(self, couleur, variete='golden'):
        super().__init__(couleur,variete)
    def __str__(self):
        return ("pomme " + super().__str__())

p = Pomme('rouge')
print(p)

En cas d'héritage multiple, `super()` *retourne le parent le plus proche* en suivant le chemin habituellement parcouru pour rechercher un attribut (MRO). Ainsi, si les méthodes de même nom s'appellent en chaîne à travers super(), on parcourt l'ensemble de l'arborescence.

In [ ]:
class A(object):
    def x(self):
        print("<A>")
        print("</A>")
    
class B1(A):
    def x(self):
        print("<B1>")
        super().x()
        print("</B1>")
    
class B2(A):
    def x(self):
        print("<B2>")
        super().x()
        print("</B2>")
    
class C(B1, B2):
    def x(self):
        print("<C>")
        super().x()
        print("</C>")

c = C()
c.x()

<img src="img-objets/heritage/heritage-super.png" width=183px>

*PYTHON 2 : avant 2.2, toutes les classes sont dites de style "ancien", avec des règles d'héritage légèrement différentes ; à partir de 2.2, pour qu'une classe soit de style "nouveau" (style par défaut de Python 3), il faut que la classe hérite de la classe object, directement ou indirectement.*


*PYTHON 2 : lors de l'apparition de la méthode super(), il fallait lui passer en argument le nom de la classe et de l'objet courants, par exemple dans la fonction x() de la classe C ci-dessus, il fallait appeler super(C,self).x().*

## A propos des auteurs

*Travail initié en 2014 dans le cadre d'une série de formations Python organisées par le réseau Devlog. Auteur principal : David Chamont. Contribution à la mise à jour pour Python 3 : Fabrice Mendes. Relecteurs : Nicolas Can, Sekou Diakite, Loic Gouarin et Christophe Halgand.*

### Mise en forme

In [ ]:
# execute this part to modify the css style
from IPython.core.display import HTML
def css_styling():
    styles = open("../../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()